# Setup

In [1]:
!pip install unsloth rich
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Using cached xformers-0.0.34-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
Using cached xformers-0.0.34-cp39-abi3-manylinux_2_28_x86_64.whl (110.8 MB)
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.26.post1
    Uninstalling xformers-0.0.26.post1:
      Successfully uninstalled xformers-0.0.26.post1
  Attempting uninstall: trl━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [xformers]
    Found existing installation: trl 0.8.6━━━━━━━━ 0/2 [xformers]
    Uninstalling trl-0.8.6:━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [xformers]
      Successfully uninstalled trl-0.8.6━━━━━━━━━━ 0/2 [xformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [trl]━━━━━━━ 1/2 [trl]
  Using cached xformers-0.0.26.post1-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.0 kB)
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached xformers-0.0.26.post1-cp311-cp311-manylinux20

In [1]:
import torch
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, TextStreamer, DataCollatorForLanguageModeling

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/chengyi/anaconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import diffusers.loaders.ip_adapter because of the following error (look up to see its traceback):
'NoneType' object has no attribute 'start'


# Initialize model

In [3]:
# Load model
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 5060 Ti. Num GPUs = 1. Max memory: 15.474 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
# Initialize LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# Initialize chat template (for instruction tuning)
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}


Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.


If we do causal language modeling, we don't need the chat template.

# Load dataset

The original data in the tutorial had 3 features: `['conversations', 'source', 'score']`

In [ ]:
"""

Example of original_data['conversations'][example]
[{'from': 'human',
  'value': 'user_message'},
 {'from': 'gpt',
  'value': 'preferred_response'}]

Example of original_data['source'][example]
'source'

Example of original_data['score'][example]
floating point 3.7 to 5.2

"""

Once we map the aply_template function, we get an extra column `text`

In [ ]:
"""

Example of original_data['text'][example]
<|im_start|>user
user_message<|im_end|>
<|im_start|>assistant
preferred_response<|im_end|>

"""

In [38]:
# Load in causal fine-tuning dataset
import pandas as pd

info_data = pd.read_csv('../Code/dataset.csv').set_index("Unnamed: 0")

In [40]:
# Check for NA values
print(f"Original dataset size: {len(info_data)}")

print("\n=== Data Quality Check ===")
print("Data types:")
print(info_data[['context', 'target_prefix', 'target_suffix']].dtypes)

print("\nNull values:")
print(info_data[['context', 'target_prefix', 'target_suffix']].isnull().sum())

print("\nEmpty strings:")
print((info_data['context'] == '').sum(), "empty contexts")
print((info_data['target_prefix'] == '').sum(), "empty prefixes")
print((info_data['target_suffix'] == '').sum(), "empty suffixes")

def clean_text(text):
    """Clean and validate text"""
    if pd.isna(text):
        return None
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    if len(text) == 0:
        return None
    return text

# Apply cleaning
info_data['context_clean'] = info_data['context'].apply(clean_text)
info_data['prefix_clean'] = info_data['target_prefix'].apply(clean_text)
info_data['suffix_clean'] = info_data['target_suffix'].apply(clean_text)

info_data_clean = info_data[
    (info_data['context_clean'].notna()) &
    (info_data['prefix_clean'].notna()) &
    (info_data['suffix_clean'].notna())
].copy()

print(f"\nDataset size after cleaning: {len(info_data_clean)}")
print(f"Removed {len(info_data) - len(info_data_clean)} bad rows")

info_data_clean['text'] = (
    info_data_clean['context_clean'] + " " + 
    info_data_clean['prefix_clean'] + 
    info_data_clean['suffix_clean']
)

dataset = Dataset.from_dict({
    'text': info_data_clean['text'].astype(str).tolist()
})

Original dataset size: 21412

=== Data Quality Check ===
Data types:
context          str
target_prefix    str
target_suffix    str
dtype: object

Null values:
context          236
target_prefix      0
target_suffix    103
dtype: int64

Empty strings:
0 empty contexts
0 empty prefixes
0 empty suffixes

Dataset size after cleaning: 21073
Removed 339 bad rows


In [47]:
import torch
torch.cuda.empty_cache()
model.gradient_checkpointing_enable()

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8, # A batch is effectively per_device_batch_size * gradient_accummulation_steps
        max_steps=5000,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        warmup_steps=100,
        output_dir="output",
        seed=0,
        save_steps=500,
        gradient_checkpointing=True,
    ),
)

trainer.train()


Map (num_proc=2): 100%|██████████| 21073/21073 [00:11<00:00, 1873.59 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,073 | Num Epochs = 4 | Total steps = 5,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
100,1.816900
200,1.779800
300,1.801800
400,1.860300
500,1.817000
600,1.861900
700,1.814700
800,1.805500
900,1.742100
1000,1.770700


TrainOutput(global_step=5000, training_loss=1.4785772811889648, metrics={'train_runtime': 20592.5612, 'train_samples_per_second': 3.885, 'train_steps_per_second': 0.243, 'total_flos': 5.958967103684813e+17, 'train_loss': 1.4785772811889648, 'epoch': 3.7941539337572365})

In [48]:
kwargs = {
    "dataset_tags": "wikimedia/wikipedia",
    "dataset": "psgs_w100",
    "dataset_args": "config: wikipedia, split: train",
    "language": "en",
    "model_name": "Llama-3.2-3B-Instruct InfoRag - Chengyi Li",
    "finetuned_from": "unsloth/Llama-3.2-3B-Instruct",
    "tasks": "causal-language-modeling",
}
trainer.push_to_hub(**kwargs)

Processing Files (3 / 3): 100%|██████████|  115MB /  115MB, 5.92MB/s  
New Data Upload: 100%|██████████|  114MB /  114MB, 5.92MB/s  


CommitInfo(commit_url='https://huggingface.co/chengyili2005/output/commit/5f68f9c439ea72e98a07d5496e30abac1d9f895b', commit_message='End of training', commit_description='', oid='5f68f9c439ea72e98a07d5496e30abac1d9f895b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chengyili2005/output', endpoint='https://huggingface.co', repo_type='model', repo_id='chengyili2005/output'), pr_revision=None, pr_num=None)

# Evaluate

In [3]:
import sys
sys.path.insert(0, '../')

from evaluation_pipeline import Evaluator

In [2]:
from datasets import load_dataset
from transformers import pipeline, BitsAndBytesConfig
import torch

# Load models
base_llm = pipeline('text-generation', model='unsloth/Llama-3.2-3B-Instruct', device_map='auto', model_kwargs={'quantization_config':BitsAndBytesConfig(load_in_8bit=True), 'dtype':torch.float16})
tuned_llm = pipeline('text-generation', model='chengyili2005/Llama-3.2-3B-Instruct-InfoRAG', device_map='auto', model_kwargs={'quantization_config':BitsAndBytesConfig(load_in_8bit=True), 'dtype':torch.float16})

# Load datasets
hotpot_dataset = load_dataset('hotpotqa/hotpot_qa', 'distractor', split='validation')
wow_dataset = load_dataset('Organika/wizard_of_wikipedia', split='test')


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.33s/it]
Device set to use cuda:0
Device set to use cuda:0
Generating test split: 100%|██████████| 2306/2306 [00:00<00:00, 173695.59 examples/s]


In [4]:
# Preprocess functions to be fed into evaluator
def preprocess_hotpot(example):
    context_parts = []
    for title, sentences in zip(example['context']['title'], example['context']['sentences']):
        context_parts.append(f"{title}: {' '.join(sentences)}")
    return {
        'question' : example['question'],
        'answer': example['answer'],
        'context': ' '.join(context_parts)
    }
def preprocess_wow(example):
    return {
        'context': example['persona'],
        'question': "You have just met the other person, who seems quite curious, and you are eager to discuss a topic with them!", # https://ar5iv.labs.arxiv.org/html/1811.01241 I got the prompt from here
        'answer': example['text']
    }

In [5]:
# Setup evaluators
hotpot_template = "Context: {context}\n\nQuestion: {question}\n\nAnswer:"
hotpot_evaluator = Evaluator(
    prompt_template=hotpot_template,
    question_key='question',
    answer_key='answer',
    context_key='context'
)
wow_template = "Context: {context}\n\nConversation: {question}\n\nAnswer:"
wow_evaluator = Evaluator(
    prompt_template=wow_template,
    question_key='question',
    answer_key='answer',
    context_key='context'
)

In [ ]:
# Run evaluation
em, f1 = hotpot_evaluator.evaluation(
    llm=base_llm,
    dataset=hotpot_dataset,
    model_name='base-model-hotpot',
    preprocess_fn=preprocess_hotpot,
    max_samples=100
)
print(f"Base HotpotQA - EM: {em:.4f}, F1: {f1:.4f}")
em, f1 = hotpot_evaluator.evaluation(
    llm=tuned_llm,
    dataset=hotpot_dataset,
    model_name='tuned-model-hotpot',
    preprocess_fn=preprocess_hotpot,
    max_samples=100
)
print(f"Tuned HotpotQA - EM: {em:.4f}, F1: {f1:.4f}")
em, f1 = wow_evaluator.evaluation(
    llm=base_llm,
    dataset=wow_dataset,
    model_name='base-model-wow',
    preprocess_fn=preprocess_wow,
    max_samples=100
)
print(f"Base WoW - EM: {em:.4f}, F1: {f1:.4f}")
em, f1 = wow_evaluator.evaluation(
    llm=tuned_llm,
    dataset=wow_dataset,
    model_name='tuned-model-wow',
    preprocess_fn=preprocess_wow,
    max_samples=100
)
print(f"Tuned WoW - EM: {em:.4f}, F1: {f1:.4f}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


HotpotQA - EM: 0.0400, F1: 0.1333
